In [1]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
from nltk.stem import WordNetLemmatizer


In [2]:
#load dtaset
from datasets import load_dataset

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

DFtrain = pd.DataFrame(load_dataset('go_emotions', split = 'train'))
DFtest = pd.DataFrame(load_dataset('go_emotions', split = 'test'))
DFval = pd.DataFrame(load_dataset('go_emotions', split = 'validation'))

No config specified, defaulting to: go_emotions/simplified
Found cached dataset go_emotions (C:/Users/AKARSH S/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)
No config specified, defaulting to: go_emotions/simplified
Found cached dataset go_emotions (C:/Users/AKARSH S/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)
No config specified, defaulting to: go_emotions/simplified
Found cached dataset go_emotions (C:/Users/AKARSH S/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


In [3]:
DFtrain.head()

,text,labels,id
0,My favourite food is anything I didn't have to cook myself.,[27],eebbqej
1,"Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj


In [4]:
DFtrain = DFtrain.append(DFval).reset_index(drop=True)

In [5]:
DFtrain = DFtrain.append(DFtest).reset_index(drop=True)

In [6]:
DF_TRAIN = DFtrain.drop(['id'],1)

In [7]:
emotions_dict = {0: "admiration", 
                 1: "amusement", 
                 2: "anger", 
                 3: "annoyance", 
                 4: "approval",
                 5: "caring",
                 6: "confusion",
                 7: "curiosity", 
                 8: "desire",
                 9: "disappointment",
                 10: "disapproval",
                 11: "disgust",
                 12: "embarrassment",
                 13: "excitement",
                 14: "fear",
                 15: "gratitude",
                 16: "grief",
                 17: "joy",
                 18: "love",
                 19: "nervousness",
                 20: "optimism",
                 21: "pride",
                 22: "realization",
                 23: "relief",
                 24: "remorse",
                 25: "sadness",
                 26: "surprise",
                 27: "neutral"
                }

In [8]:
DF_TRAIN['labels_str'] = DF_TRAIN['labels'].apply(lambda tpl: [emotions_dict.get(x) for x in tpl])

In [9]:
target_for_values = {
    'admiration': 'admiration', 
    'gratitude': 'admiration', 
    'anger': 'anger',
    'annoyance':'anger',
    'approval': 'approval',
    'relief': 'approval',
    'curiosity': 'curiosity',
    'confusion': 'curiosity',
    'desire': 'curiosity',
    'disgust': 'disgust',
    'embarrassment': 'disgust',
    'joy': 'joy',
    'amusement': 'joy',
    'excitement': 'joy',
    'love': 'love',
    'caring': 'love',
    'nervousness': 'fear',
    'fear': 'fear',
    'optimism': 'optimism',
    'pride': 'pride',
    'disappointment': 'disappointment', 
    'disapproval': 'disappointment', 
    'surprise': 'surprise',
    'realization': 'surprise',
    'neutral': 'neutral',
    'grief': 'grief',
    'sadness': 'grief',
    'remorse': 'grief'}


In [10]:
DF_TRAIN['labels_merged'] = DF_TRAIN['labels_str'].apply(lambda tpl: [target_for_values.get(x) for x in tpl])

In [11]:
DF_TRAIN.head()

,text,labels,labels_str,labels_merged
0,My favourite food is anything I didn't have to cook myself.,[27],[neutral],[neutral]
1,"Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead",[27],[neutral],[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,[2],[anger],[anger]
3,To make her feel threatened,[14],[fear],[fear]
4,Dirty Southern Wankers,[3],[annoyance],[anger]


In [12]:
DF_TRAIN['Len_of_classes'] = DF_TRAIN['labels'].apply(lambda x: len(x))

In [13]:
DF_TRAIN['labels_str'] = DF_TRAIN['labels_str'].astype(str).str.replace(r'\[|\]|,', '')
DF_TRAIN['labels_merged'] = DF_TRAIN['labels_merged'].astype(str).str.replace(r'\[|\]|,', '')

In [14]:
DF_LABEL = DF_TRAIN.assign(labels_merged=DF_TRAIN['labels_merged'].str.split()).explode('labels_merged')


In [15]:
DF_LABEL = DF_LABEL.reset_index(drop=True)

In [16]:
DF_LABEL = DF_LABEL.astype(str).drop_duplicates(subset=['text', 'labels', 'labels_str'], keep='first')

In [17]:
DF_LABELCLASS = DF_LABEL.reset_index(drop=True)

In [18]:
DF_LABELCLASS = DF_LABELCLASS.drop('Len_of_classes', 1)

In [19]:
from wordcloud import WordCloud, STOPWORDS
wr = ' '.join(DF_LABELCLASS['labels_merged']).lower().split()
sw = set(STOPWORDS)
words = [wr for wr in wr if wr not in sw]

In [20]:
wf = {}
for wrd in wr:
    if wrd not in wf:
        wf[wrd] = 1
    else:
        wf[wrd] += 1

In [21]:
from nltk import FreqDist
from collections import Counter

def top_fty(doc, k=50):
    dc_wrd = DF_LABELCLASS['text']
    fty = Counter([word.lower() for word in dc_wrd])
    wrd = fty.most_common(k)

    return wrd

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import f1_score
import nltk,re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import wordpunct_tokenize
stp_wrd = set(stopwords.words('english'))

from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
nltk.download('omw-1.4')

ps = nltk.PorterStemmer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to C:\Users\AKARSH
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\AKARSH
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\AKARSH
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\AKARSH
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [23]:
DF_LABELCLASS = DF_LABELCLASS.drop(['labels','labels_str'],1)

In [24]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

In [29]:
def preprocessing_stemming(texts): 
    Tlist=[]
    for sentence in texts:
        if isinstance(sentence, str) == False:
            sentence = ""
            
        fltSntnc=[]
        # change to lowercase
        sentence = sentence.lower() 
        # whitespace removel
        sentence = sentence.strip() 
        # extra spaces and tabs are removed
        sentence = re.sub('\s+', ' ', sentence) 
        # markup of html removed
        sentence = re.compile('<.*?>').sub('', sentence) 
        # word [NAME] is frequently appearing so that will be removed
        sentence = re.sub(r'\[name\]', '', sentence) 
        # emoticons, symbols & pictographs, transport & map symbols, flags
        emoji_pattern = re.compile("[" u"\U0001F600-\U0001F64F" 
                                   u"\U0001F300-\U0001F5FF" 
                                   u"\U0001F680-\U0001F6FF" 
                                   u"\U0001F1E0-\U0001F1FF" 
                                                       "]+", flags=re.UNICODE)
        sentence = emoji_pattern.sub(r'', sentence)


    
        tokenizer = WordPunctTokenizer()
        
        for word in tokenizer.tokenize(sentence):
            if(not word.isnumeric()) and (len(word)>2) and (word not in stop_words):  
                  fltSntnc.append(stemmer.stem(word))   # Stem and add to filtered list
        final_string = " ".join(fltSntnc) # final string of cleaned words
 
        Tlist.append(final_string)
    
        
    return Tlist

In [30]:
X_train, X_test = train_test_split(DF_LABEL, test_size = 0.10, stratify = DF_LABEL['labels_merged'], random_state=324)# Applying the preprocessing on the test and train split 

In [32]:
X_train['text'] = preprocessing_stemming(X_train["text"])
X_test['text'] = preprocessing_stemming(X_test["text"])

In [33]:
tfidf = TfidfVectorizer(lowercase= True, max_features=1000, stop_words=list(ENGLISH_STOP_WORDS))

In [34]:
tfidf.fit(X_train.text)

TfidfVectorizer(max_features=1000,
                stop_words=['some', 'in', 'afterwards', 'as', 'something', 'us',
                            'give', 'hereafter', 'namely', 'yours', 'couldnt',
                            'and', 'go', 'these', 'those', 'whenever',
                            'against', 'third', 'its', 'now', 'twelve',
                            'within', 'ie', 'besides', 'somehow', 'ever',
                            'interest', 'mostly', 'you', 'whence', ...])

In [35]:
TFIDFtrain = tfidf.transform(X_train.text)
TFIDFtest  = tfidf.transform(X_test.text)

In [36]:
model_LR = LogisticRegression(solver='sag',penalty='l2', C=10)

In [37]:
model_LR.fit(TFIDFtrain, X_train.labels_merged)

LogisticRegression(C=10, solver='sag')

In [38]:
trainPrediction = model_LR.predict(TFIDFtrain)
testPrediction = model_LR.predict(TFIDFtest)

In [39]:
# f1 score on train data
LRSCR = f1_score(y_true= X_train.labels_merged, y_pred= trainPrediction, average='micro')
LRSCR

0.5628762198253724

In [40]:
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,
                                                      max_features=1000,
                                                      stop_words= ENGLISH_STOP_WORDS)),
                            ('model', LogisticRegression(solver='sag',penalty='l2', C=10))])

In [41]:
pipeline.fit(X_train.text, X_train.labels_merged)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
           

In [42]:
pipeline.classes_

array(["'admiration'", "'anger'", "'approval'", "'curiosity'",
       "'disappointment'", "'disgust'", "'fear'", "'grief'", "'joy'",
       "'love'", "'neutral'", "'optimism'", "'pride'", "'surprise'"],
      dtype=object)

In [43]:
#Testing the Intent for the given value of input
text = ["i want to book a hotel in florida"]

In [44]:
pipeline.predict(text)

array(["'neutral'"], dtype=object)

In [45]:
text1 = ["WHY THE FUCK IS BAYLESS ISOING	"]

In [46]:
pipeline.predict(text1)

array(["'anger'"], dtype=object)

In [47]:
import pickle
new_file=open('testmodel.pckl','wb')
pickle.dump(pipeline,new_file)
new_file.close()